In [88]:
import soundfile # to read audio file
import numpy as np
import librosa # to extract speech features
import glob
import os
import pickle # to save model after training
import pandas as pd
# import sklearn
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split # for splitting training and testing
# from sklearn.neural_network import MLPClassifier # multi-layer perceptron model
# from sklearn.metrics import accuracy_score, f1_score # to measure how good we are



In [89]:
#load data from csv files
X_train = pd.read_csv(os.path.join("./prepared-data/", "X_train.csv")).to_numpy()
X_test = pd.read_csv(os.path.join("./prepared-data/", "X_test.csv")).to_numpy()
y_train = pd.read_csv(os.path.join("./prepared-data/", "y_train.csv")).to_numpy()
y_test = pd.read_csv(os.path.join("./prepared-data/", "y_test.csv")).to_numpy()

X_train = X_train[:,1:]
X_test = X_test[:,1:]
y_train = y_train[:,1]
y_test = y_test[:,1]

In [90]:
# load the model from disk
filename = "trained-model/ada_classifier.model"
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print('accuracy', result)

accuracy 0.7055555555555556


In [94]:
# # load the model from disk
# X_train_cnn = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
# X_test_cnn = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
# filename = "trained-model/cnn_classifier"
# loaded_cnn_model = keras.models.load_model(filename)
# score, acc = loaded_cnn_model.evaluate(X_test_cnn, y_test)
# ans = loaded_cnn_model(prepX[0].reshape(1,-1))
# print(ans)

In [91]:
def extract_feature(file_name, **kwargs):
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")

    if chroma or contrast:
        X, sample_rate = librosa.load(file_name)
        stft = np.abs(librosa.stft(X))
    result = np.array([])

    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))

    return result


def data_transformer(data_path):
    prep_X = []
    for file in glob.glob(data_path):
        basename = os.path.basename(file)
        features = extract_feature(file, mfcc=True, chroma=True, mel=True)
        prep_X.append(features)
    return prep_X

data_path = "test_audio/*.wav"
prepX = data_transformer(data_path)

In [93]:
loaded_model.predict([prepX[0]])

array([1])